In [1]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# connect to the Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive/

Mounted at /content/gdrive
 1.pdf
'2020-10-16_Лекция_2 курс.pdf'
 20201019_175328.jpg
 20201019_180028.jpg
 20201019_180033.jpg
'20201019_180037 (1).jpg'
 20201019_180037.jpg
'20201019_180043 (1).jpg'
 20201019_180043.jpg
 20201019_180100.jpg
 20201019_181155.jpg
'2020-10-19_Лекция_2 курс.pdf'
 20201028_230347.jpg
 20201102_102210.jpg
 20201102_102215.jpg
 20201103_225547.jpg
 20201103_225551.jpg
 20201103_225557.jpg
 20201103_225600.jpg
 20201103_225621.jpg
 20210512_171545.mp4
 20210512_173245.mp4
'2.1, 2.2. ОДУ 1-го порядка, не разрешенные относительно производной (1).gdoc'
'2.1, 2.2. ОДУ 1-го порядка, не разрешенные относительно производной.gdoc'
'2.5. Теорема существования и единственности ....gdoc'
'2.7. Уравнения Лагранжа и Клеро. (1).gdoc'
'2.7. Уравнения Лагранжа и Клеро..gdoc'
 2.jpg
'3.6. Уравнения, допускающие понижение порядка (1).gdoc'
'3.6. Уравнения, допускающие понижение порядка (2).gdoc'
'3.6. Уравнения, допускающие понижение порядка.gdoc'
'4-5. 10. Овеществление КР

In [ ]:
import torch
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/gdrive/MyDrive/yolov5/weights/best.pt')  # local repo

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-3-27 torch 1.10.0+cu111 CPU

Fusing layers... 
YOLOv5s summary: 232 layers, 7284276 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
%cd /content/gdrive/MyDrive/yolov5/

/content/gdrive/MyDrive/yolov5


In [ ]:
from webcamdetect import *
import queue


# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
bbox = ''
# Load the model and set the hyperparameters up
cudnn.benchmark = True
model.conf = 0.25  # confidence threshold (0-1)
model.iou = 0.45  # NMS IoU threshold (0-1)
model.classes = None #[0, 15, 16]  # (optional list) filter by class, i.e. = [0, 15, 16] for persons, cats and dogs

my_queue = queue.Queue()
def storeInQueue(f):
  def wrapper(*args):
    my_queue.put(f(*args))
  return wrapper

@storeInQueue
def detection(model, frame, bbox_array):
  result = model(frame)
  
  Class_name = result.pandas().xyxy[0].name
  Confidence = result.pandas().xyxy[0].confidence
  colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(Class_name))]
  Xmin = result.pandas().xyxy[0].xmin
  Ymin = result.pandas().xyxy[0].ymin
  Xmax = result.pandas().xyxy[0].xmax
  Ymax = result.pandas().xyxy[0].ymax
  
  _, _, h, w = result.s
  img_height, img_width, _ = frame.shape
  
  width_ratio = img_width/w
  height_ratio = img_height/h

  for i in range(0, len(Class_name)):
    xmin = int(Xmin[i] + 0.5)
    ymin = int(Ymin[i] + 0.5)
    xmax = int(Xmax[i] + 0.5)
    ymax = int(Ymax[i] + 0.5)

    xmin = int(xmin * width_ratio)
    ymin = int(ymin * height_ratio)
    xmax = int(xmax * width_ratio)
    ymax = int(ymax * height_ratio)

    bbox_array = cv2.rectangle(bbox_array, (xmin, ymin), (xmax, ymax), colors[i], 2)
    bbox_array = cv2.putText(bbox_array, Class_name[i],
                        (xmin, ymin - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        colors[i], 2)
  bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
  # convert overlay of bbox into bytes
  bbox_bytes = bbox_to_bytes(bbox_array)
  
  # update bbox so next frame gets new overlay
  # bbox = bbox_bytes
  return bbox_bytes



while True:
  js_reply = video_frame(label_html, bbox)
  if not js_reply:
    break
  
  # convert JS response to OpenCV Image
  frame = js_to_image(js_reply["img"])

  # create transparent overlay for bounding box
  bbox_array = np.zeros([480,640,4], dtype=np.uint8)

  t1 = threading.Thread(target=detection, args=(model, frame, bbox_array))
  t1.start()
  #t1.join()
  my_data = my_queue.get()
  bbox = my_data

<IPython.core.display.Javascript object>